#### Мета роботи: реалізувати програму для виконання обчислень функцій Лаґерра та їх перетворень

In [1]:
import numpy as np
import ipywidgets
import matplotlib.pyplot as plt

##### Для обчислення функцій Лаґерра використовують формули:     
\begin{aligned}
    l_n(t)=\frac{2n-1-\sigma t}{n}l_{n-1}(t)-\frac{n-1}{n} l_{n-2}(t), \quad n \geq 2, \\
    l_0(t)=\sqrt{\sigma} e^{-\frac{\beta}{2}t}, \quad l_1(t)=\sqrt{\sigma}(1-\sigma t) e^{-\frac{\beta}{2}t}.
\end{aligned}
##### За умовою вважатимемо B=2, o=4 за замовчуванням

In [2]:
def lagger(t,n,b=2,o=4):
    l0=np.sqrt(o)*np.exp(-b*t/2)
    l1=np.sqrt(o)*(1-o*t)*np.exp(-b*t/2)
    if n==0:
        return l0
    elif n==1:
        return l1
    else:
        for i in range(2,n+1):
            l2= (2*i-1-o*t)*l1/i-(i-1)*l0/i
            l0,l1=l1,l2
        return l2           

In [3]:
lagger(1,20)

-0.11094613260729669

#### Далі реалізуємо функцію для табуляції функції Лаґерра на заданому відрізку. Це нам знадобиться потім при реалізації графіків.

In [4]:
def tabulate_lagger(n, b, o, T, k):
    h=T/k
    x=[i*h for i in range(k+1)]
    y=[lagger(x1,n,b,o) for x1 in x]
    return x,y

def print_tabulation(x,y):
    for i in range(len(x)):
        print(round(x[i],5),y[i],sep='\t')

In [5]:
x,y=tabulate_lagger(6,2,4,10,100)
print_tabulation(x,y)

0.0	1.9999999999999998
0.1	-0.7199603933429105
0.2	-0.7641813065507715
0.3	0.02330791918637709
0.4	0.726198401745203
0.5	0.9974059737496637
0.6	0.812360642831173
0.7	0.30467967036618093
0.8	-0.3384117298915382
0.9	-0.9381832335195615
1.0	-1.3570663829879872
1.1	-1.51161084951959
1.2	-1.3728750240045944
1.3	-0.9594041604635353
1.4	-0.3264258153428452
1.5	0.44626032029685936
1.6	1.2660819017103342
1.7	2.0381991360213045
1.8	2.6741879580862298
1.9	3.098693359575141
2.0	3.254061699155888
2.1	3.1030999686892775
2.2	2.630191911912317
2.3	1.8410414014013066
2.4	0.7613248042313814
2.5	-0.5654744349646362
2.6	-2.0807970180884503
2.7	-3.7140815857331826
2.8	-5.385990208349137
2.9	-7.0116279673766435
3.0	-8.503631277231161
3.1	-9.775029620427063
3.2	-10.741812403296521
3.3	-11.32515630435539
3.4	-11.453288660298062
3.5	-11.06297918000762
3.6	-10.100665783998124
3.7	-8.523230903596227
3.8	-6.298452453709581
3.9	-3.405159247078064
4.0	0.16687582098624532
4.1	4.417269599757404
4.2	9.335553788645857


#### Проведемо обчислювальний експеримент: для N=20 знайти точку T>0, щоб  $|l_n(T)| < eps = 10^{-3} $ для усіх $n \in [0, N]$

In [6]:
def experiment(T,b,o,eps=1e-3,N=20):
    t=0
    h=1/10
    while t<=T:
        flag=True
        for n in range(1,N+1):
            if abs(lagger(t,n,b,o))>eps:
                flag=False
                break     
        if flag:
            return t
        else:
            t+=h

In [7]:
r=experiment(100,2,4)
print(r)

79.09999999999978


##### Реалізуємо функцію для наближеного обчислення інтегралів, використовуючи метод прямокутників, за формулою: $$f_k = \int_{0}^{T} f(t)l_k(t)e^{-ot} \, dt ,\    k\in[0, N]$$

In [8]:
def integral(f,a,b,k):
    in1=0
    h=(b-a)/k
    for i in range(k):
        x=h*i+h/2
        in1+=f(x)
    return in1*h

def integral_main(f, a,b,eps=0.00001):
    k=10
    prev=integral(f,a,b,k)
    k*=2
    cur=integral(f,a,b,k)
    while abs(prev-cur)>eps:
        k*=2
        prev,cur=cur,integral(f,a,b,k)
    return cur

##### Нижче допоміжна функція, яка рахує значення інтегралу в точці

In [9]:
def func(t):
    return lagger(t,2,2,4)*lagger(t,3,2,4)*np.exp(-2*t)

In [10]:
print(integral_main(func,0,100))

-1.4901121472831865e-06


##### Для заданої функції виконати Перетворення Лаґерра, а саме знайти коефіцієнти $f^N := (f_0, f_1,..., f_N)^T$ при N=10

\begin{cases}
    cos(t-\pi/4)+1-sin(t+\pi/6),\ &t\in [0,2\pi] \\
    0,\ &t>=2\pi \\
\end{cases}

In [11]:
def f_cos(t):
    if t>=2*np.pi:
        return 0
    else:
        return np.cos(t-np.pi/4)+1-np.sin(t+np.pi/6)


#### Перетворення Лаґерра

In [12]:
def lagger_transformation(f,T,N=10):
    fk=[]
    
    for i in range(N+1):
        def int_func(t):
            return f(t)*lagger(t,i,2,4)*np.exp(-2*t)
        fk.append(integral_main(int_func,0,T))
    return fk    

In [13]:
h=lagger_transformation(f_cos,r)
print(h)

[0.7591458311093235, -0.18601182936288674, 0.04109031538239029, -0.018732421028574066, 0.012394863182049813, -0.005453122986364942, 0.0008142670798417326, 0.0008139930946952234, -0.0008291950825886384, 0.00022703627424913006, 0.0003420671473524219]


#### Отримуємо вектор коефіцієнтів h, для якого реалізуємо функцію оберненого перетворення Лаґерра

In [14]:
def reverse_lagger(h,x):
    sum=0
    for i in range(len(h)):
        sum+=h[i]*lagger(x,i,2,4)
    return sum

#### Побудуємо графіки функцій Лаґерра

In [15]:
def plot_lagger(T, N, b=2, o=4, k=100):
    fig = plt.figure(figsize=(10, 10))
    ax = fig.gca()
    for n in range(N+1):
        x, y = tabulate_lagger(n, b, o, T, k)
        ax.plot(x, y, label=f"n={n}", linewidth=2.0, alpha=0.7)
    
    ax.set_xlabel("t")
    ax.set_ylabel("l(t)")
    ax.set_title("Lagger polynomials")
    fig.legend(loc='lower center', ncol=5)
    plt.show()

In [16]:
s0=ipywidgets.IntSlider(value=5, min=1,max=30,step=1,description="N")
s01=ipywidgets.FloatSlider(value=1, min=0.1,max=30,step=0.1,description="T")
w0=ipywidgets.interactive(plot_lagger, T=s01, N=s0, b=ipywidgets.fixed(2),o=ipywidgets.fixed(4),k=ipywidgets.fixed(100))
display(w0)

interactive(children=(FloatSlider(value=1.0, description='T', max=30.0, min=0.1), IntSlider(value=5, descripti…

#### Для раніше заданої функції виконаємо пряме та обернене перетворення Лаґерра і побудуємо графік функцій. Також дослідимо точність перетворення залежно від значення N

In [17]:
print(f_cos(2),'\t',reverse_lagger(h,2))

0.7693088735788127 	 0.7694580608437979


In [18]:
def plot_f_cos(a,b,N):
    h=lagger_transformation(f_cos,r,N)
    x=np.linspace(a,b,100)
    y=[f_cos(i) for i in x]
    z=[reverse_lagger(h,i) for i in x]
    
    plt.figure(figsize=(10, 10))
    plt.plot(x,y,x,z)
    plt.show()

In [19]:
s=ipywidgets.IntSlider(value=1, min=1,max=20,step=1,description="n")
w1=ipywidgets.interactive(plot_f_cos, a=ipywidgets.fixed(1),b=ipywidgets.fixed(6),N=s)
display(w1)

interactive(children=(IntSlider(value=1, description='n', max=20, min=1), Output()), _dom_classes=('widget-int…

##### Висновок: при виконанні даного завдання мені вдалось реалізувати функції для обчислення функцій, прямого та оберненого перетворення Лаґерра. Важливою складовою роботи була взаємодія з matplotlib та ipywidgets Також мені вдалось оцінити зручність роботи у форматі Notebook і ознайомитись із онлайн редактором LaTeX.